# pythreejs

[pythreejs](https://github.com/jovyan/pythreejs) allows 3D interactive graphs in a notebook.

[documentation](http://threejs.org/docs/index.html#Manual/Introduction/Creating_a_scene) [source](https://github.com/jovyan/pythreejs) [installation](https://github.com/jovyan/pythreejs#installation) [tutorial](https://github.com/jovyan/pythreejs/blob/master/examples/Examples.ipynb) [gallery](http://threejs.org/) 

In [1]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

## Installation

*pythreejs* requires local installation.

```
pip install pythreejs
jupyter nbextension enable --py pythreejs
```

## example

In [2]:
from pythreejs import *
import numpy as np
from IPython.display import display
from ipywidgets import HTML, Text
from traitlets import link, dlink

In [3]:
ball = Mesh(geometry=SphereGeometry(radius=1), material=MeshLambertMaterial(color='red'), position=[2,1,0])
scene = Scene(children=[ball, AmbientLight(color='#777777'), make_text('Hello World!', height=.6)])
c = PerspectiveCamera(position=[0, 5, 5],
                      up=[0, 0, 1],
                      children=[DirectionalLight(color='white', position=[3, 5, 1], intensity=0.5)])
renderer = Renderer(camera=c, scene=scene, controls=[OrbitControls(controlling=c)])
display(renderer)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, position=(3.0, 5.0, 1.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)),), position=(0.0, 5.0, 5.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 0.0, 1.0)), controls=[OrbitControls(controlling=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, position=(3.0, 5.0, 1.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)),), position=(0.0, 5.0, 5.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 0.0, 1.0)))], scene=Scene(children=(Mesh(geometry=SphereGeometry(), material=MeshLambertMaterial(alphaMap=None, aoMap=None, color='red', emissiveMap=None, envMap=None, lightMap=None, map=None, specularMap=None), position=(2.0, 1.0, 0.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)), AmbientLight(color='#777777', quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)), Sprite(center=(0.5, 0.5), material=SpriteMaterial(map=TextTexture(repeat=(1.0, 1.0), size=100, squareTexture=False, string='Hello World!')), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 0.6, 1.0), up=(0.0, 1.0, 0.0))), fog=None, overrideMaterial=None, quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)), shadowMap=WebGLShadowMap())

In [4]:
nx, ny = (20, 20)
xmax=1
x = np.linspace(-xmax, xmax, nx)
y = np.linspace(-xmax, xmax, ny)
xx, yy = np.meshgrid(x, y)
z = xx ** 2 - yy ** 2
#z[6,1] = float('nan')
surf_g = SurfaceGeometry(z=list(z[::-1].flat), 
                         width=2 * xmax,
                         height=2 * xmax,
                         width_segments=nx - 1,
                         height_segments=ny - 1)


surf = Mesh(geometry=surf_g, material=MeshLambertMaterial(map=height_texture(z[::-1], 'YlGnBu_r')))

surfgrid = SurfaceGrid(geometry=surf_g, material=LineBasicMaterial(color='black'))
hover_point = Mesh(geometry=SphereGeometry(radius=0.05), material=MeshLambertMaterial(color='hotpink'))
scene = Scene(children=[surf, surfgrid, hover_point, AmbientLight(color='#777777')])
c = PerspectiveCamera(position=[0, 3, 3], up=[0, 0, 1], 
                      children=[DirectionalLight(color='white', position=[3, 5, 1], intensity=0.6)])

pickable_objects = Group()
pickable_objects.add(surf)

click_picker = Picker(controlling = pickable_objects, event='dblclick')
hover_picker = Picker(controlling = pickable_objects, event='mousemove')
renderer = Renderer(camera=c, scene = scene, controls=[OrbitControls(controlling=c), click_picker, hover_picker])

def f(change):
    value = change['new']
    print('Clicked on %s' % value)
    point = Mesh(geometry=SphereGeometry(radius=0.05), 
                 material=LambertMaterial(color='red'),
                 position=value)
    scene.children = list(scene.children) + [point]

click_picker.observe(f, names=['point'])

link((hover_point, 'position'), (hover_picker, 'point'))

h = HTML()
def g(change):
    h.value = 'Green point at (%.3f, %.3f, %.3f)' % tuple(change['new'])
g({'new': hover_point.position})
hover_picker.observe(g, names=['point'])
display(h)
display(renderer)

HTML(value='Green point at (0.000, 0.000, 0.000)')

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(3.0, 5.0, 1.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)),), position=(0.0, 3.0, 3.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 0.0, 1.0)), controls=[OrbitControls(controlling=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(3.0, 5.0, 1.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)),), position=(0.0, 3.0, 3.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 0.0, 1.0))), Picker(controlling=Group(children=(Mesh(geometry=SurfaceGeometry(attributes={'position': <BufferAttribute shape=(400, 3), dtype=float32>, 'index': <BufferAttribute shape=(2166,), dtype=uint16>, 'normal': <BufferAttribute shape=(400, 3), dtype=float32>, 'uv': <BufferAttribute shape=(400, 2), dtype=float32>}), material=MeshLambertMaterial(alphaMap=None, aoMap=None, emissiveMap=None, envMap=None, lightMap=None, map=DataTexture(data=array([[[ 66, 182, 196, 255],
        [ 36, 152, 193, 255],
        [ 32, 118, 179, 255],
        ...,
        [ 32, 118, 179, 255],
        [ 36, 152, 193, 255],
        [ 66, 182, 196, 255]],

       [[115, 200, 189, 255],
        [ 66, 182, 196, 255],
        [ 40, 156, 193, 255],
        ...,
        [ 40, 156, 193, 255],
        [ 66, 182, 196, 255],
        [115, 200, 189, 255]],

       [[165, 220, 183, 255],
        [109, 198, 190, 255],
        [ 66, 182, 196, 255],
        ...,
        [ 66, 182, 196, 255],
        [109, 198, 190, 255],
        [165, 220, 183, 255]],

       ...,

       [[165, 220, 183, 255],
        [109, 198, 190, 255],
        [ 66, 182, 196, 255],
        ...,
        [ 64, 181, 196, 255],
        [109, 198, 190, 255],
        [165, 220, 183, 255]],

       [[115, 200, 189, 255],
        [ 66, 182, 196, 255],
        [ 40, 156, 193, 255],
        ...,
        [ 40, 156, 193, 255],
        [ 64, 181, 196, 255],
        [115, 200, 189, 255]],

       [[ 66, 182, 196, 255],
        [ 36, 152, 193, 255],
        [ 32, 118, 179, 255],
        ...,
        [ 32, 118, 179, 255],
        [ 36, 152, 193, 255],
        [ 66, 182, 196, 255]]], dtype=uint8), repeat=(1.0, 1.0)), specularMap=None), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)),), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)), event='dblclick', object=None), Picker(controlling=Group(children=(Mesh(geometry=SurfaceGeometry(attributes={'position': <BufferAttribute shape=(400, 3), dtype=float32>, 'index': <BufferAttribute shape=(2166,), dtype=uint16>, 'normal': <BufferAttribute shape=(400, 3), dtype=float32>, 'uv': <BufferAttribute shape=(400, 2), dtype=float32>}), material=MeshLambertMaterial(alphaMap=None, aoMap=None, emissiveMap=None, envMap=None, lightMap=None, map=DataTexture(data=array([[[ 66, 182, 196, 255],
        [ 36, 152, 193, 255],
        [ 32, 118, 179, 255],
        ...,
        [ 32, 118, 179, 255],
        [ 36, 152, 193, 255],
        [ 66, 182, 196, 255]],

       [[115, 200, 189, 255],
        [ 66, 182, 196, 255],
        [ 40, 156, 193, 255],
        ...,
        [ 40, 156, 193, 255],
        [ 66, 182, 196, 255],
        [115, 200, 189, 255]],

       [[165, 220, 183, 255],
        [109, 198, 190, 255],
        [ 66, 182, 196, 255],
        ...,
        [ 66, 182, 196, 255],
        [109, 198, 190, 255],
        [165, 220, 183, 255]],

       ...,

       [[165, 220, 183, 255],
        [109, 198, 190, 255],
        [ 66, 182, 196, 255],
        ...,
        [ 64, 181, 196, 255],
        [109, 198, 190, 255],
        [165, 220, 183, 255]],

       [[115, 200, 189, 255],
        [ 66, 182, 196, 255],
        [ 40, 156, 193, 255],
        ...,
        [ 40, 156, 193, 255],
        [ 64, 181, 196, 255],
        [115, 200, 189, 255]],

       [[ 66, 182, 196, 255],
        [ 36, 152, 193, 255],
        [ 32, 118, 179, 255],
        ...,
